<a href="https://colab.research.google.com/github/joshjarvey/Coursera_Capstone/blob/master/Capstone_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim
import requests
import json

In [0]:
  #This data was gathered from an open public source via the City of Madison's Database:http://data-cityofmadison.opendata.arcgis.com/datasets/neighborhood-plans
  #Since the database is an ARCGIS database, it only contained SHAPE data.
  #The data was downloaded as a "SHAPE" file and converted to latitude and longitude using the free tool here:https://mygeodata.cloud/converter/shp-to-latlong

  #read in the .csv data, and drop the columns that are not needed for the analysis of this project
df_msn_neighborhood = pd.read_csv('/tmp/Neighborhood_Plans.csv').drop(['OBJECTID_1','OBJECTID','Status','Year','SHAPE_Leng','Link','ShapeSTAre','ShapeSTLen'], axis=1)

  #rename the columns for easier referencing in the project.
df_msn_neighborhood.rename(columns={'Y':'Latitude','X':'Longitude','PlanName':'Neighborhood'}, inplace=True)

  #cleaning the names of the neighborhood values 
df_msn_neighborhood["Neighborhood"]= df_msn_neighborhood["Neighborhood"].str.replace('Neighborhood', '')
df_msn_neighborhood["Neighborhood"]= df_msn_neighborhood["Neighborhood"].str.replace('Plan', '')
df_msn_neighborhood


In [3]:
#getting the lat/long coordinates of Madison, using the geolocater module, for the map module below
address = 'Madison, WI'

geolocator = Nominatim(user_agent="madison_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

The geograpical coordinate of Madison are 43.074761, -89.3837613.


In [0]:

  #Opening and reading in foursquare API credentials file - JSON format
with open('/tmp/foursquare_credentials.json') as f:
    credentials = json.load(f) 
CLIENT_ID = credentials['CLIENT_ID']
CLIENT_SECRET = credentials['CLIENT_SECRET']
VERSION = '20180605' # Foursquare API version

  #building a list of URLs that will be passed to the GET API to gather grocery stores
url_list=[]
for i in range(len(df_msn_neighborhood)):
     #extracting the lat and long values from the neighborhood dataframe
  neighborhood_latitude = df_msn_neighborhood.loc[i,'Latitude']
  neighborhood_longitude = df_msn_neighborhood.loc[i,'Longitude']
  neighborhood_name = df_msn_neighborhood.loc[i,'Neighborhood']

    #compiling the URL
  limit = 100
  radius = 4000
  query = 'Grocery Store'
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit,
    query
    )
    #once compiled, then add to the list
  url_list.append(url)


In [0]:
store_data=[]

  #loop through the urls list and make each API call. Within the loop, a nested loop will parse out the required data to build the dataframe
for j in range(len(url_list)):
  results = requests.get(url_list[j]).json()
  
    #Looping through each unique element returned within the JSON to build a nested list of relevant data per the API call. 
    #Note: --> ['items'][i]['venue']  <- the index of the "items" is what needs to be looped through to gather each records detials in the JSON.
  for i in range(len(results['response']['groups'][0]['items'])):
      #grabbing the type of venue from the JSON
    store_type = results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
      #grabbing the store name from the JSON
    store_name = results['response']['groups'][0]['items'][i]['venue']['name']
      #grabbing the formatted address & city/state from the JSON
    store_address = results['response']['groups'][0]['items'][i]['venue']['location']['formattedAddress'][0] + ' ,' + results['response']['groups'][0]['items'][0]['venue']['location']['formattedAddress'][1]
      #grabbing the latitude and longitude from the JSON
    store_lat = results['response']['groups'][0]['items'][i]['venue']['location']['labeledLatLngs'][0]['lat']
    store_lng = results['response']['groups'][0]['items'][i]['venue']['location']['labeledLatLngs'][0]['lng']
  
      #build a temp list of the items. 
    store_data.append([store_type,store_name,store_address,store_lat,store_lng])

  #create a dataframe using the store data list extracted
df_grocery_store = pd.DataFrame(columns=['Store Type', 'Store Name','Store Address', 'Store Latitude', 'Store Longitude'], data=store_data)



In [93]:
df_grocery_store[df_grocery_store['Store Name'] == 'Woodman\'s Food Market']

##note, there are overlaping radius selections with current 4000m search radius. Need to consider how to not double count unique stores.

,Store Type,Store Name,Store Address,Store Latitude,Store Longitude
2,Supermarket,Woodman's Food Market,"3817 Milwaukee StMadison, WI 53704",43.097609,-89.318843
34,Supermarket,Woodman's Food Market,"3817 Milwaukee StMadison, WI 53703",43.097609,-89.318843
91,Supermarket,Woodman's Food Market,"3817 Milwaukee StMadison, WI 53704",43.097609,-89.318843
107,Grocery Store,Woodman's Food Market,"711 S Gammon Rd. (at Watts Rd.)Madison, WI 53711",43.049238,-89.501573
209,Supermarket,Woodman's Food Market,"3817 Milwaukee StMadison, WI 53714",43.097609,-89.318843
231,Grocery Store,Woodman's Food Market,"711 S Gammon Rd. (at Watts Rd.)Madison, WI 53711",43.049238,-89.501573
244,Supermarket,Woodman's Food Market,"3817 Milwaukee StMadison, WI 53704",43.097609,-89.318843
362,Supermarket,Woodman's Food Market,"3817 Milwaukee StMadison, WI 53718",43.097609,-89.318843


In [80]:
#create map of Madison using latitude and longitude values
map_madison = folium.Map(location=[latitude, longitude], zoom_start=12)

    #add grocery store markers radius, 1 mile per definition of a food desert.
for lat, lng, label in zip(df_grocery_store['Store Latitude'], df_grocery_store['Store Longitude'], df_grocery_store['Store Name']):
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=1610,
        color='#556b2f',
        fill=True,
        fill_color='#556b2f',
        fill_opacity=0.05,
        parse_html=False).add_to(map_madison)  

    #add neighborhood markers to map
for lat, lng, label in zip(df_msn_neighborhood['Latitude'], df_msn_neighborhood['Longitude'], df_msn_neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madison)  

    #add grocery store markers to map
for lat, lng, label in zip(df_grocery_store['Store Latitude'], df_grocery_store['Store Longitude'], df_grocery_store['Store Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madison)  

  #generate map
map_madison
